---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

<a target="_blank" href="https://colab.research.google.com/github/LuisAngelMendozaVelasco/Applied_Data_Science_with_Python_Specialization/blob/main/Applied_Social_Network_Analysis_in_Python/Week4/Labs/Assignment 4.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>

In [1]:
%%bash
# If running in Google Colab

# mkdir data
# wget https://raw.githubusercontent.com/LuisAngelMendozaVelasco/Applied_Data_Science_with_Python_Specialization/main/Applied_Social_Network_Analysis_in_Python/Week4/Labs/data/A4_graphs -P ./data
# wget https://raw.githubusercontent.com/LuisAngelMendozaVelasco/Applied_Data_Science_with_Python_Specialization/main/Applied_Social_Network_Analysis_in_Python/Week4/Labs/data/email_prediction.txt -P ./data
# wget https://raw.githubusercontent.com/LuisAngelMendozaVelasco/Applied_Data_Science_with_Python_Specialization/main/Applied_Social_Network_Analysis_in_Python/Week4/Labs/data/Future_Connections.csv -P ./data

# Assignment 4

In [2]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

In [3]:
nx.__version__

'1.11'

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [4]:
P1_Graphs = pickle.load(open('./data/A4_graphs', 'rb'))
P1_Graphs

`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:

* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [5]:
def graph_identification():  
    # Your Code Here
    return ['PA', 'SW_L', 'SW_L', 'PA', 'SW_H'] # Your Answer Here

graph_identification()

['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be working with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [6]:
G = nx.read_gpickle('./data/email_prediction.txt')
print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.

Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

Example:
    
    1       1.0
    2       0.0
    5       0.8
    8       1.0
        ...
    996     0.7
    1000    0.5
    1001    0.0
    Length: 252, dtype: float64

In [7]:
from sklearn.ensemble import GradientBoostingClassifier

def salary_predictions():
    # Your Code Here
    input_data = pd.DataFrame()
    target = nx.get_node_attributes(G, 'ManagementSalary')
    degree_centrality = nx.degree_centrality(G)
    closeness_centrality = nx.closeness_centrality(G)
    betweenness_centrality = nx.betweenness_centrality(G)

    for node in target.keys():
        row_features = pd.DataFrame([[degree_centrality[node], closeness_centrality[node],
                                      betweenness_centrality[node], target[node]]], index=[node],
                                      columns=['Degree centrality', 'Closeness centrality', 'Betweenness centrality', 'Target'])
        input_data = pd.concat([input_data, row_features], axis=0)

    train_data = input_data[~input_data['Target'].isnull()]
    test_data = input_data[input_data['Target'].isnull()]
    clf = GradientBoostingClassifier()
    clf.fit(train_data.drop('Target', axis=1).values, train_data['Target'].values)
    preds = clf.predict_proba(test_data.drop('Target', axis=1).values)[:, 1]  
     
    return pd.Series(preds, index=test_data.index) # Your Answer Here

salary_predictions()

1       0.042805
2       0.986968
5       0.986379
8       0.097363
14      0.264127
          ...   
992     0.008780
994     0.008545
996     0.008545
1000    0.009177
1001    0.006089
Length: 252, dtype: float64

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [8]:
future_connections = pd.read_csv('./data/Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head()

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.

Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

Example:
    
    (107, 348)    0.35
    (542, 751)    0.40
    (20, 426)     0.55
    (50, 989)     0.35
                ...
    (939, 940)    0.15
    (555, 905)    0.35
    (75, 101)     0.65
    Length: 122112, dtype: float64

In [9]:
from sklearn.ensemble import GradientBoostingClassifier

def new_connections_predictions():   
    # Your Code Here
    future_connections['pref_attachment'] = [list(nx.preferential_attachment(G, [node_pair]))[0][2]
                                             for node_pair in future_connections.index]
    future_connections['comm_neighbors'] = [len(list(nx.common_neighbors(G, node_pair[0], node_pair[1]))) 
                                            for node_pair in future_connections.index]

    train_data = future_connections[~future_connections['Future Connection'].isnull()]
    test_data = future_connections[future_connections['Future Connection'].isnull()]
    cl = GradientBoostingClassifier()
    cl.fit(train_data[['pref_attachment', 'comm_neighbors']].values, train_data['Future Connection'].values)
    preds = cl.predict_proba(test_data[['pref_attachment', 'comm_neighbors']].values)[:, 1]
    
    return pd.Series(preds, index=test_data.index) # Your Answer Here

new_connections_predictions()

(107, 348)    0.031823
(542, 751)    0.012931
(20, 426)     0.543026
(50, 989)     0.013104
(942, 986)    0.013103
                ...   
(165, 923)    0.013183
(673, 755)    0.013103
(939, 940)    0.013103
(555, 905)    0.012931
(75, 101)     0.017730
Length: 122112, dtype: float64